<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>CU18_Infraestructuras_eventos</font></center>

<font color='steelblue'><b>Citizenlab Data Science Methodology > II - Data Processing Domain </font>
***
> # <font color='steelblue'> <b>05.- Data Collection</font>

<font color='steelblue'>Data Collection is the process to obtain and generate (if required) necessary data to model the problem.</font>

### 15. Agregar datos meteorológicos por distritos y año

* Estimar variables en distritos
* Agrupar promedios anuales de los distritos en las variables meteorológicas
* Eliminando variables que no se usarán en modelos







<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Settings" data-toc-modified-id="Settings-1"><font color="green">Settings</font></a></span></li><li><span><a href="#Data-Load" data-toc-modified-id="Data-Load-2"><font color="green">Data Load</font></a></span></li><li><span><a href="#ETL-Processes" data-toc-modified-id="ETL-Processes-3">ETL Processes</a></span><ul class="toc-item"><li><span><a href="#Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files" data-toc-modified-id="Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files-3.1">Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files</a></span></li></ul></li><li><span><a href="#Synthetic-Data-Generation" data-toc-modified-id="Synthetic-Data-Generation-4">Synthetic Data Generation</a></span></li><li><span><a href="#Fake-Data-Generation" data-toc-modified-id="Fake-Data-Generation-5">Fake Data Generation</a></span></li><li><span><a href="#Open-Data" data-toc-modified-id="Open-Data-6">Open Data</a></span></li><li><span><a href="#Data-Save" data-toc-modified-id="Data-Save-7"><font color="green">Data Save</font></a></span></li><li><span><a href="#Main-Conclusions" data-toc-modified-id="Main-Conclusions-8"><font color=" #2874a6 ">Main Conclusions</font></a></span></li><li><span><a href="#Main-Actions" data-toc-modified-id="Main-Actions-9"><font color=" #2874a6 ">Main Actions</font></a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Acciones-done" data-toc-modified-id="Acciones-done-9.0.1"><font color=" #2874a6 ">Acciones done</font></a></span></li><li><span><a href="#Acctions-to-perform" data-toc-modified-id="Acctions-to-perform-9.0.2"><font color=" #2874a6 ">Acctions to perform</font></a></span></li></ul></li></ul></li></ul></div>

## <font color='green'>Settings</font>

### Packages to use

- {tcltk} para selección interactiva de archivos locales
- {sf} para trabajar con georeferenciación
- {gstat} para cálculos geoestadísticos
- {readr} para leer y escribir archivos csv
- {dplyr} para explorar datos
- {stringr} para manipulación de cadenas de caracteres
- {tidyr} para quitar perdidos


In [55]:
library(readr)
library(dplyr)
library(sf)
library(gstat)
library(stringr)
library(tidyr)


### Paths

In [2]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

If there are more than one input file, make as many sections as files to import.

Instrucciones
- Los ficheros de entrada del proceso están siempre en Data/Input/.   
- Si hay más de un fichero de entrada, se crean tantos objetos iFile_xx y file_data_xx como ficheros de entrada (xx número correlativo con dos dígitos, rellenar con ceros a la izquierda)

1. Datos meteorológicos de estaciones

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [ ]:
# file_data_01 <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [10]:
iFile_01 <- "CU_18_05_14_aemet.csv"
file_data_01 <- paste0(iPath, iFile_01)

if(file.exists(file_data)){
    cat("Se leerán datos del archivo: ", file_data_01)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leer<U+00E1>n datos del archivo:  Data/Input/CU_18_05_14_aemet.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [23]:
data_01 <- read_csv(file_data_01)


Rows: 4293 Columns: 22
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (10): indicativo, nombre, provincia, prec, horatmin, horatmax, dir, hor...
dbl  (11): altitud, tmed, tmin, tmax, velmedia, racha, sol, presMax, presMin...
date  (1): fecha

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


Estructura de  los datos:

In [24]:
glimpse(data_01)

Rows: 4,293
Columns: 22
$ fecha       <date> 2022-01-01, 2022-01-02, 2022-01-03, 2022-01-04, 2022-01-0~
$ indicativo  <chr> "2462", "2462", "2462", "2462", "2462", "2462", "2462", "2~
$ nombre      <chr> "PUERTO DE NAVACERRADA", "PUERTO DE NAVACERRADA", "PUERTO ~
$ provincia   <chr> "MADRID", "MADRID", "MADRID", "MADRID", "MADRID", "MADRID"~
$ altitud     <dbl> 1894, 1894, 1894, 1894, 1894, 1894, 1894, 1894, 1894, 1894~
$ tmed        <dbl> 10.6, 9.8, 7.8, 1.6, -3.0, -5.6, -4.0, -2.8, 0.0, 1.6, 1.0~
$ prec        <chr> "0,0", "0,0", "0,0", "24,2", "13,9", "0,0", "0,2", "0,0", ~
$ tmin        <dbl> 7.4, 6.0, 4.2, -3.9, -4.2, -8.3, -6.6, -4.6, -3.0, 0.3, -0~
$ horatmin    <chr> "00:10", "20:40", "06:20", "23:59", "03:10", "06:40", "03:~
$ tmax        <dbl> 13.7, 13.5, 11.3, 7.0, -1.8, -2.8, -1.3, -0.9, 3.0, 3.0, 2~
$ horatmax    <chr> "Varias", "13:30", "Varias", "01:40", "Varias", "00:00", "~
$ dir         <chr> "22", "22", "27", "33", "32", "35", "36", "33", "31", "33"~
$ velmedia    <d

Muestra de datos:

In [25]:
slice_head(data_01, n = 5)

fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin,X,Y
<date>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,...,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
2022-01-01,2462,PUERTO DE NAVACERRADA,MADRID,1894,10.6,"0,0",7.4,00:10,13.7,...,3.9,12.2,22:30,8.1,823.3,11,821.8,06,-4.010556,40.79306
2022-01-02,2462,PUERTO DE NAVACERRADA,MADRID,1894,9.8,"0,0",6.0,20:40,13.5,...,2.5,11.1,00:00,6.0,823.5,Varias,821.5,Varias,-4.010556,40.79306
2022-01-03,2462,PUERTO DE NAVACERRADA,MADRID,1894,7.8,"0,0",4.2,06:20,11.3,...,3.1,15.3,22:50,7.6,821.5,00,814.7,24,-4.010556,40.79306
2022-01-04,2462,PUERTO DE NAVACERRADA,MADRID,1894,1.6,"24,2",-3.9,23:59,7.0,...,7.8,23.6,18:30,3.3,814.7,00,803.6,17,-4.010556,40.79306
2022-01-05,2462,PUERTO DE NAVACERRADA,MADRID,1894,-3.0,"13,9",-4.2,03:10,-1.8,...,4.2,19.2,00:30,0.0,809.5,11,806.4,Varias,-4.010556,40.79306


2. Datos de contornos distritos

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [ ]:
# file_data_02 <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [17]:
iFile_02 <- "CU_18_05_03_distritos_geo.json"
file_data_02 <- paste0(iPath, iFile_02)

if(file.exists(file_data)){
    cat("Se leerán datos del archivo: ", file_data_02)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leer<U+00E1>n datos del archivo:  Data/Input/CU_18_05_03_distritos_geo.json

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [18]:
data_02 <- st_read(file_data_02)


Reading layer `CU_18_05_03_distritos_geo' from data source 
  `/Users/emilio.lcano/academico/gh_repos/__transferencia/citizenlab/CitizenLab-Research-and-Development/casos_urjc/notebooks/II_data_processing/18_infraestructuras/Data/Input/CU_18_05_03_distritos_geo.json' 
  using driver `GeoJSON'
Simple feature collection with 247 features and 2 fields
Geometry type: GEOMETRY
Dimension:     XY
Bounding box:  xmin: -4.579006 ymin: 39.8848 xmax: -3.052983 ymax: 41.16584
Geodetic CRS:  WGS 84


Estructura de  los datos:

In [19]:
glimpse(data_02)

Rows: 247
Columns: 3
$ CMUN     <chr> "001", "002", "003", "004", "005", "005", "005", "005", "005"~
$ CDIS     <chr> "01", "01", "01", "01", "01", "02", "03", "04", "05", "01", "~
$ geometry <POLYGON [arc_degree]> POLYGON ((-3.64502 41.12129..., POLYGON ((-3~


Muestra de datos:

In [20]:
data_02 |> tibble() |> slice_head(n = 5)

CMUN,CDIS,geometry
<chr>,<chr>,<POLYGON [arc_degree]>
001,01,POLYGON ((-3.64502 41.12129...
002,01,"POLYGON ((-3.503032 40.526,..."
003,01,POLYGON ((-3.808664 40.8921...
004,01,POLYGON ((-4.00197 40.25642...
005,01,POLYGON ((-3.361691 40.4762...


## ETL Processes

### Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files

Se han importado en el apartado Data Load anterior:

* Datos diarios meteorológicos por estación
* Contornos de distritos censales

Incluir apartados si procede para: Extracción de datos (select, filter), Transformación de datos, (mutate, joins, ...). Si es necesario tratar datos perdidos, indicarlo también en NB 09.2

Si no aplica: Estos datos no requieren tareas de este tipo.

#### Data transform

* Transformar precipitación a numérica
* Ip es inapreciable, se sustituye por 0

* Corregir geometrías distritos y eliminar vacíos

In [37]:
tdata_01 <- data_01 |>
    mutate(prec = str_replace(prec, "Ip", "0"),
        prec = as.numeric(str_replace_all(prec, ",", ".")))

In [ ]:
tdata_02 <- data_02 |> st_make_valid()
tdata_02 <- tdata_02 |> filter(!st_is_empty(tdata_02))

#### Data extract

* Seleccionar variables temperatura, precipitaciones y presión
* Agrupar por año

In [48]:
tdata_01_grouped <- tdata_01 |>
    select(nombre, tmed, prec, velmedia, presMax, X, Y) |>
    group_by(nombre, X, Y) |>
    summarise(across(everything(), ~mean(.x, na.rm = TRUE)))

`summarise()` has grouped output by 'nombre', 'X'. You can override using the
`.groups` argument.


In [49]:
glimpse(tdata_01_grouped)

Rows: 13
Columns: 7
Groups: nombre, X [13]
$ nombre   <chr> "ARANJUEZ", "BUITRAGO DEL LOZOYA", "COLMENAR VIEJO", "GETAFE"~
$ X        <dbl> -3.546111, -3.613611, -3.765000, -3.722222, -3.555556, -3.724~
$ Y        <dbl> 40.06722, 41.00694, 40.69611, 40.29944, 40.46667, 40.45167, 4~
$ tmed     <dbl> 16.552089, 14.076370, 15.084110, 17.388219, 16.394795, NaN, 1~
$ prec     <dbl> 0.9991597, 0.9513889, 1.3838356, 1.0742466, 1.0624658, 0.1272~
$ velmedia <dbl> 1.823416, 1.583904, 2.436712, 3.276986, 2.982692, 1.018182, 2~
$ presMax  <dbl> NaN, 907.2604, 906.5734, 948.9011, 953.0038, NaN, 941.1178, 9~


In [50]:
tdata_01_grouped

nombre,X,Y,tmed,prec,velmedia,presMax
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ARANJUEZ,-3.546111,40.06722,16.552089,0.9991597,1.823416,NaN
BUITRAGO DEL LOZOYA,-3.613611,41.00694,14.076370,0.9513889,1.583904,907.2604
COLMENAR VIEJO,-3.765000,40.69611,15.084110,1.3838356,2.436712,906.5734
GETAFE,-3.722222,40.29944,17.388219,1.0742466,3.276986,948.9011
MADRID AEROPUERTO,-3.555556,40.46667,16.394795,1.0624658,2.982692,953.0038
"MADRID, CIUDAD UNIVERSITARIA",-3.724167,40.45167,NaN,0.1272727,1.018182,NaN
"MADRID, CUATRO VIENTOS",-3.786111,40.37556,17.074521,1.2260274,2.823288,941.1178
"MADRID, RETIRO",-3.678056,40.41194,17.066197,1.2483333,2.054104,943.1645
PUERTO ALTO DEL LE<U+00D3>N,-4.141944,40.70639,12.927711,1.9791908,4.246286,851.4212


#### Data transform

* Convertir a objeto `sf` para poder hacer las estimaciones

In [61]:
tdata_01_grouped_sf <- st_as_sf(tdata_01_grouped, coords = c("X", "Y"), crs = 4326)

#### Data transform

* Estimar valores medios de cada distrito con los datos puntuales
* Unir a la tabla

In [84]:


tmed <- idw(tmed ~ 1, 
          locations = tdata_01_grouped_sf |> drop_na(tmed), 
          newdata = tdata_02)

prec <- idw(prec ~ 1, 
          locations = tdata_01_grouped_sf |> drop_na(prec), 
          newdata = tdata_02)

velmedia <- idw(velmedia ~ 1, 
          locations = tdata_01_grouped_sf |> drop_na(velmedia), 
          newdata = tdata_02)
presMax <- idw(presMax ~ 1, 
          locations = tdata_01_grouped_sf |> drop_na(presMax), 
          newdata = tdata_02)


[inverse distance weighted interpolation]
[inverse distance weighted interpolation]
[inverse distance weighted interpolation]
[inverse distance weighted interpolation]


In [91]:
tdata_02_meteo <- tdata_02 |>
    mutate(tmed = tmed$var1.pred,
            prec = prec$var1.pred,
            velmedia = velmedia$var1.pred,
            presMax = presMax$var1.pred) |>
    st_drop_geometry()

In [92]:
glimpse(tdata_02_meteo)

Rows: 246
Columns: 6
$ CMUN     <chr> "001", "002", "003", "004", "005", "005", "005", "005", "005"~
$ CDIS     <chr> "01", "01", "01", "01", "01", "02", "03", "04", "05", "01", "~
$ tmed     <dbl> 12.26152, 16.01595, 13.00403, 16.21605, 15.97587, 15.98693, 1~
$ prec     <dbl> 1.9519845, 0.9757789, 1.9142270, 1.1881407, 0.9800614, 0.9633~
$ velmedia <dbl> 1.932702, 2.996452, 2.725458, 2.553527, 2.991905, 3.037137, 3~
$ presMax  <dbl> 905.5722, 946.7792, 881.6706, 932.5134, 945.8788, 946.8818, 9~


In [93]:
tdata_02_meteo |> slice_head(n = 5)

CMUN,CDIS,tmed,prec,velmedia,presMax
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
001,01,12.26152,1.9519845,1.932702,905.5722
002,01,16.01595,0.9757789,2.996452,946.7792
003,01,13.00403,1.9142270,2.725458,881.6706
004,01,16.21605,1.1881407,2.553527,932.5134
005,01,15.97587,0.9800614,2.991905,945.8788


####

## Synthetic Data Generation

No aplica

## Fake Data Generation

No aplica

## Open Data

Los datos originales fueron descargados de fuentes abiertas:

* Meteorológicas de AEMET
* Geográficas (distritos) del INE

## <font color='green'>Data Save</font>

Este proceso, puede copiarse y repetirse en aquellas partes del notebbok que necesiten guardar datos.
Recuerde cambiar las cadenas añadida del fichero para diferenciarlas

<font color='tomato'> Identificamos los datos a guardar</font>

In [96]:
data_to_save <- tdata_02_meteo


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_05".
* Número de la tarea que lo genera, por ejemplo "_01"
* En caso de generarse varios ficheros en la misma tarea, llevarán _01 _02 ... después
* Nombre: identificativo de "properData", por ejemplo "_zonasgeo"
* Extensión del archivo

Ejemplo: "CU_04_05_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas)

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 05

In [94]:
caso <- "CU_18"
proceso <- '_05'
tarea <- "_15"
archivo <- ""
proper <- "_distritos_meteo"
extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar datos por datos_xx si es necesario

In [ ]:
# file_save <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save, path_out)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [97]:
file_save <- paste0(caso, proceso, tarea, archivo, proper, extension) 
path_out <- paste0(oPath, file_save)
write_csv(data_to_save, path_out)

cat('File saved as: ')
path_out

File saved as: 

[1] "Data/Output/CU_18_05_15_distritos_meteo.csv"

#### Copia del fichero a Input

Si el archivo se va a usar en otros notebooks, copiar a la carpeta Input

In [98]:
path_in <- paste0(iPath, file_save)
file.copy(path_out, path_in, overwrite = TRUE)

[1] TRUE

## <font color=' #2874a6 '>Main Conclusions</font>

<font color='steelblue'>List and describe the general conclusions of the analysis carried out​.</font>

### Prerequisites

This working code needs the following conditions:

* For using the interactive selection of file, the {tcltk} package must be installed. It is not needed in production.
* The {sf}, {gstat}, {readr}, {dplyr}, {stringr} and {tidyr} packages must be installed. 
* The data paths `Data/Input` and `Data/Output` must exist (relative to the notebook path)

### Configuration Management

This notebook has been tested with the following versions of R and packages. It cannot be assured that later versions work in the same way:
* R 4.2.2
* tcltk 4.2.2
* sf 1.0.9
* gstat 2.1.0
* readr 2.1.3
* dplyr 1.0.10
* stringr 1.5.0
* tidyr 1.3.0

### Data structures

#### Objeto `data`

- Hay 246 filas con las variables:
    * CMUN
    * CDIS
    * tmed
    * prec
    * velmedia
    * presMax


In [105]:
colnames(tdata_02_meteo)

[1] "CMUN"     "CDIS"     "tmed"     "prec"     "velmedia" "presMax"

#### Observaciones generales sobre los datos

* Se han agregado los datos totales porque se tiene un año. Si se usan más años en los modelos, el agrupamiento debe ser de la granularidad de la que haya datos.

### Consideraciones para despliegue en piloto

* Si llegan datos reales se deberían descargar los datos de las fechas que coincidan

### Consideraciones para despliegue en producción

* Se deben crear los procesos ETL en producción necesarios para que los datos de entrada estén actualizados

## <font color=' #2874a6 '>Main Actions</font>

#### <font color=' #2874a6 '>Acciones done</font>

<font color='steelblue'>Indicate the actions that have been carried out in this process</font>

- Se han xxxx

#### <font color=' #2874a6 '>Acctions to perform</font>

<font color='steelblue'>Indicate the actions that must be carried out in subsequent processes</font>

- Se deben unir a los datos agregados de distritos del resto de variables

## <font color='RED'>CODE TO DEPLOY (PILOT)</font>

A continuación se incluirá el código que deba ser llevado a despliegue para producción, dado que se entiende efectúa operaciones necesarias sobre los datos en la ejecución del prototipo

<font color=' red '><b>Description</b></font>

- No hay nada que desplegar en el piloto, ya que estos datos son estáticos o en todo caso cambian con muy poca frecuencia, altamente improbable durante el proyecto.


<font color=' red '><b>CODE</b></font>

In [ ]:
# incluir código